In [13]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from implicit import als, lmf
from scipy.sparse import csr_matrix

In [2]:
!ls ../data/raw/steam-reviews-dataset/

reviews-1-115.csv       reviews-13495-13500.csv reviews-4575-6805.csv
reviews-11265-13495.csv reviews-13500-13537.csv reviews-6805-9035.csv
reviews-115-1230.csv    reviews-13537-27075.csv reviews-9035-11265.csv
reviews-1230-2345.csv   reviews-2345-4575.csv


In [17]:
%%time
# chunk_paths = [path for path in pathlib.Path("../data/raw/steam-reviews-dataset/").glob("*.csv")]
# dfs = []
# for i in chunk_paths:
#     dfs.append(pd.read_csv(i))
# df = pd.concat(dfs)
df = pd.read_csv("../data/raw/steam-reviews-dataset/reviews-1-115.csv")
df.head()

CPU times: user 3.27 s, sys: 550 ms, total: 3.82 s
Wall time: 4.14 s


,steamid,appid,voted_up,votes_up,votes_funny,weighted_vote_score,playtime_forever,playtime_at_review,num_games_owned,num_reviews,review,unix_timestamp_created,unix_timestamp_updated
0,76561198107294407,10,True,0,0,0.0,8549,8549,3,1,best gameplay ever created,1619038050,1619038050
1,76561198011733201,10,True,0,0,0.0,25381,25381,74,1,Timeless. Thank you volvo.,1619013754,1619013754
2,76561198168961276,10,True,0,0,0.0,681,616,26,6,this game literally made me love guns and fps ...,1619008681,1619008681
3,76561198957877160,10,True,0,0,0.0,2071,1990,5,1,If this gets a rework in 2021 it's going to be...,1619003706,1619003706
4,76561199050314447,10,True,0,0,0.0,36718,36718,3,1,ok,1618994842,1618994842


In [18]:
print(df.shape)

(1460621, 13)


In [19]:
df["unix_timestamp_created"] = pd.to_datetime(df["unix_timestamp_created"], unit="s")
df["unix_timestamp_updated"] = pd.to_datetime(df["unix_timestamp_updated"], unit="s")
# df["unix_timestamp_created"] = 

In [20]:
df["voted_up"] = df["voted_up"].astype(int)

In [21]:
dataset = df[["steamid", "appid", "voted_up", "playtime_at_review", "unix_timestamp_created"]]
steamid_to_index = {steam_id: index for index, steam_id in enumerate(dataset["steamid"].values)}
app_id_to_index = {app_id: index for index, app_id in enumerate(dataset["appid"].values)}
dataset.loc[:, "user_id"] = dataset["steamid"].map(steamid_to_index)
dataset.loc[:, "item_id"] = dataset["appid"].map(app_id_to_index)
# dataset.to_csv("../data/interim/steam-games-review-interim.csv")

/var/folders/58/q1f7wcd51gjfk1jks4vsk0z80000gn/T/ipykernel_21855/1354114264.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "user_id"] = dataset["steamid"].map(steamid_to_index)
/var/folders/58/q1f7wcd51gjfk1jks4vsk0z80000gn/T/ipykernel_21855/1354114264.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "item_id"] = dataset["appid"].map(app_id_to_index)


Add train-test-split on datetime

## Explicit feedback

In [10]:
%%time
# positive_reviews = dataset.query("voted_up == 1")
user_item_matrix = csr_matrix(
    (dataset["voted_up"].to_numpy(), (dataset["user_id"].to_numpy(), dataset["item_id"].to_numpy())),
    # shape=(dataset["steamid"].nunique(), dataset["appid"].nunique()),
)

CPU times: user 27.6 ms, sys: 16.9 ms, total: 44.5 ms
Wall time: 43.3 ms


In [12]:
%%time
# Initialize and train the ALS model
model = als.AlternatingLeastSquares(factors=50, regularization=0.1, random_state=21122024) #factors is the number of latent factors
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 45 s, sys: 952 ms, total: 46 s
Wall time: 47.2 s


In [60]:
model.save("../models/als-all-reviews-f-50-r-0.1-21-12-2024.npz")

Calculate mAp@10, calculate diversity and novelty. Cold start on games or users? 

In [ ]:
# Recommend items for a user
user_id = 0
recommendations = model.recommend(user_id, user_item_matrix[user_id])
print(recommendations)

# Get similar items
item_id = 0
similar = model.similar_items(item_id)
print(similar)

## Implicit feedback

In [22]:
user_item_matrix_implicit = csr_matrix(
    (dataset["playtime_at_review"].to_numpy(), (dataset["user_id"].to_numpy(), dataset["item_id"].to_numpy())),
    # shape=(dataset["steamid"].nunique(), dataset["appid"].nunique()),
)

In [24]:
model_implicit = lmf.LogisticMatrixFactorization(factors=30, regularization=0.1, random_state=21122024)
model_implicit.fit(user_item_matrix_implicit)

  0%|          | 0/30 [00:00<?, ?it/s]